In [26]:
import os
import pickle as pkl

import matplotlib.pyplot as plt
import numpy as np
import torch

In [70]:
dataset1 = 'COCO_train'
dataset2 = 'DOTA_train'

path = os.path.join('../inter-classes-matrix', dataset1 + 'x' + dataset2)

dataset_size1 = 80
dataset_size2 = 16

In [71]:
heatmap = np.zeros((dataset_size1, dataset_size2))

for i in range(dataset_size1):
    for j in range(dataset_size2):
        with open(os.path.join(path, dataset1 + str(i) + 'x' + dataset2 + str(j) + '_cov.pkl'), 'rb') as f:
            heatmap[i, j] = pkl.load(f)

In [65]:
heatmap = torch.tensor(heatmap)/100
heatmap = torch.softmax(heatmap, dim=0)

In [18]:
for j in range(dataset_size2):
    heatmap[:, j] = (heatmap[:, j] - heatmap[:, j].min()) / (heatmap[:, j].max() - heatmap[:, j].min())

TypeError: softmax() received an invalid combination of arguments - got (Tensor), but expected one of:
 * (Tensor input, int dim, torch.dtype dtype, *, Tensor out)
 * (Tensor input, name dim, *, torch.dtype dtype)


In [13]:
for i in range(dataset_size1):
    heatmap[i, :] = (heatmap[i, :] - heatmap[i, :].min()) / (heatmap[i, :].max() - heatmap[i, :].min())

In [72]:
%matplotlib qt

# plt.imshow(heatmap.cpu().numpy())
plt.imshow(heatmap)